In [1]:
import os

import pandas as pd

In [2]:
cwd = os.getcwd()   #Current working directory

In [3]:
gravity_dict = {}
time_periods = ['2000_2004', '2005_2009', '2010_2014', '2015_2019']
for item in time_periods:
    gravity_dict[item] = pd.read_csv(os.path.join(cwd, 'data', f'release_2.1_{item}.csv'))

df_trade_balance = pd.read_csv(os.path.join(cwd, 'data', 'DOT_07-12-2022 22-28-04-74_timeSeries.csv'))

c:\Users\Daniel\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (31,35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df_trade_balance = df_trade_balance[(df_trade_balance['Indicator Name'].isin([
    'Goods, Value of Exports, Free on board (FOB), US Dollars',
    'Goods, Value of Imports, Free on board (FOB), US Dollars'])) & \
    (df_trade_balance['Attribute'] == 'Value')
]

In [5]:
df_trade_balance.insert(0, 'Index_col', df_trade_balance['Country Name'] + ' - ' + df_trade_balance['Counterpart Country Name'])

In [6]:
years = df_trade_balance.columns[-20: -1]

df_trade_balance_final = df_trade_balance.pivot(
    index = 'Index_col',
    columns = 'Indicator Name',
    values = '2000'
)
df_trade_balance_final.columns = df_trade_balance_final.columns.get_level_values(0)
df_trade_balance_final.index.name = None
df_trade_balance_final.columns.name = None
df_trade_balance_final.index += ' - 2000'

for year in years:
    df_temp = df_trade_balance.pivot(
        index = 'Index_col',
        columns = 'Indicator Name',
        values = year
    )
    df_temp.columns = df_temp.columns.get_level_values(0)
    df_temp.index.name = None
    df_temp.columns.name = None
    df_temp.index += f' - {year}'
    df_trade_balance_final = pd.concat([df_trade_balance_final, df_temp])

df_trade_balance_final.reset_index(inplace = True)

In [7]:
df_gravity = gravity_dict['2000_2004'].copy()
df_gravity.index = df_gravity['country_o'] + ' - ' + df_gravity['country_d'] + ' - ' + df_gravity['year'].astype(str)
df_gravity.drop(df_gravity.columns[:7], axis = 1, inplace = True)

for item in time_periods[1:]:
    df_temp = gravity_dict['2000_2004'].copy()
    df_temp.index = df_temp['country_o'] + ' - ' + df_temp['country_d'] + ' - ' + df_temp['year'].astype(str)
    df_temp.drop(df_temp.columns[:7], axis = 1, inplace = True)
    df_gravity = pd.concat([df_gravity, df_temp])

df_gravity.reset_index(inplace = True)

In [8]:
df_merge = df_trade_balance_final.merge(df_gravity)

In [9]:
df_merge[~df_merge.isna()].drop_duplicates()

,index,"Goods, Value of Exports, Free on board (FOB), US Dollars","Goods, Value of Imports, Free on board (FOB), US Dollars",colony_of_destination_ever,colony_of_origin_ever,colony_ever,common_colonizer,common_legal_origin,contiguity,distance,...,sanction_imposition,sanction_imposition_trade,gdp_wdi_cur_o,gdp_wdi_cap_cur_o,gdp_wdi_const_o,gdp_wdi_cap_const_o,gdp_wdi_cur_d,gdp_wdi_cap_cur_d,gdp_wdi_const_d,gdp_wdi_cap_const_d
0,Albania - Algeria - 2000,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1597.835592,...,0.0,0.0,3.480355e+09,1126.683318,6.933726e+09,2244.631092,5.478607e+10,1764.888222,1.104352e+11,3557.579333
4,Albania - Angola - 2000,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,5732.503227,...,0.0,0.0,3.480355e+09,1126.683318,6.933726e+09,2244.631092,9.129595e+09,556.836318,3.599840e+10,2195.630582
8,Albania - Anguilla - 2000,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,8105.697930,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Albania - Antigua and Barbuda - 2000,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,8082.268359,...,0.0,0.0,3.480355e+09,1126.683318,6.933726e+09,2244.631092,8.301588e+08,10920.842688,1.002264e+09,13184.909241
16,Albania - Argentina - 2000,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,11686.594487,...,0.0,0.0,3.480355e+09,1126.683318,6.933726e+09,2244.631092,2.842038e+11,7708.100996,3.032295e+11,8224.112827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375472,Zimbabwe - United States - 2004,40018946,36719706,0.0,0.0,0.0,0.0,0.0,0.0,14174.351106,...,0.0,0.0,5.805598e+09,482.998411,1.244016e+10,1034.962988,1.221373e+13,41712.801068,1.384606e+13,47287.593772
375476,Zimbabwe - Uruguay - 2004,NaN,79439,0.0,0.0,0.0,0.0,0.0,0.0,8571.876439,...,0.0,0.0,5.805598e+09,482.998411,1.244016e+10,1034.962988,1.368633e+10,4120.556611,2.806385e+10,8449.209089
375480,Zimbabwe - Vanuatu - 2004,294,NaN,0.0,0.0,0.0,0.0,0.0,0.0,13902.827147,...,0.0,0.0,5.805598e+09,482.998411,1.244016e+10,1034.962988,3.649969e+08,1788.087167,5.219731e+08,2557.099850
375484,Zimbabwe - Vietnam - 2004,4140528,2712,0.0,0.0,0.0,0.0,0.0,0.0,9153.933561,...,0.0,0.0,5.805598e+09,482.998411,1.244016e+10,1034.962988,4.542785e+10,546.909606,7.936215e+10,955.447267
